In [ ]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /home/inflaton/code/CrediNews


In [ ]:
from dotenv import find_dotenv, load_dotenv

found_dotenv = find_dotenv(".env")

if len(found_dotenv) == 0:
    found_dotenv = find_dotenv(".env.example")
print(f"loading env vars from: {found_dotenv}")
load_dotenv(found_dotenv, override=True)

loading env vars from: /home/inflaton/code/CrediNews/.env


True

In [ ]:
# !pip install wandb
import os
os.getenv("WANDB_API_KEY", "empty")[-5:]

'f541b'

In [ ]:
import ray

# If running on a cluster uncomment use the line below instead
# ray.init(address="auto", log_to_driver=False)

ray.shutdown()
ray.init(log_to_driver=True, ignore_reinit_error=True)

2025-02-22 11:26:37,905	INFO worker.py:1841 -- Started a local Ray instance.


Python version:,3.11.11
Ray version:,2.42.1


In [ ]:
import os
from transformers import DataCollatorWithPadding, AutoTokenizer, AutoModelForSequenceClassification

# Change these as needed.
model_name = "bert-base-uncased"

# Triggers tokenizer download to cache
tokenizer = AutoTokenizer.from_pretrained(model_name)
print("Downloading and caching pre-trained model")

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Triggers model download to cache
AutoModelForSequenceClassification.from_pretrained(model_name)

2025-02-22 11:26:39.561327: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-22 11:26:39.568810: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740194799.577332  375902 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740194799.579587  375902 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-22 11:26:39.588130: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

## Training

With everything now downloaded and cached, we can now set up our training function. Our training function defines the training execution for a single hyperparameter configuration. For now we pull these hyperparameters from a config argument, but we’ll see later how this is passed in.

First we get our datasets- we only use the first half of the dev dataset for validation, and leave the rest of testing:

In [ ]:
from datasets import load_dataset, concatenate_datasets, Dataset

def get_datasets(config="original"):
    datasets = load_dataset(
        "csv",
        data_files={
            "train": [
                "dataset/train_data_1.csv",
                "dataset/train_data_2.csv",
                "dataset/train_data_3.csv",
                "dataset/train_data_4.csv",
            ],
            "test": "dataset/test_data.csv",
            "rewritten_train": [
                "dataset/rewritten_train_data_1.csv",
                "dataset/rewritten_train_data_2.csv",
                "dataset/rewritten_train_data_3.csv",
                "dataset/rewritten_train_data_4.csv",
            ],
            "rewritten_test": "dataset/rewritten_test_data.csv",
        },
    )

    # Tokenize the dataset
    def tokenize_function(example):
        return tokenizer(example["processed_full_content"], padding="max_length", truncation=True)

    tokenized_datasets = datasets.map(tokenize_function, batched=True)
    
    train_dataset = tokenized_datasets["train"]
    eval_dataset = tokenized_datasets["test"]

    return train_dataset, eval_dataset

get_datasets()

Map:   0%|          | 0/6050 [00:00<?, ? examples/s]

(Dataset({
     features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 54441
 }),
 Dataset({
     features: ['label', 'full_content', 'processed_full_content', 'input_ids', 'token_type_ids', 'attention_mask'],
     num_rows: 6050
 }))

### Checkpointing

We also need to add extra functionality for *checkpointing*. After every epoch of training, we need to save our training state. This is crucial for Population Based Training since it allows us to continue training from where we left off even when hyperparameters are perturbed. The Huggingface Trainer provides functionality to save and load from a checkpoint, but we do have to make some modifications to integrate this with Ray Tune checkpointing and to checkpoint after every epoch. The first step is to subclass the Trainer from the transformers library. Ray Tune provides this [TuneTransformerTrainer](https://github.com/ray-project/ray/blob/master/python/ray/tune/examples/pbt_transformers/trainer.py) subclass which we utilize. Take a look at the class- we see that it handles reporting evaluation metrics to Tune, checkpointing everytime evaluate is called, and even a way to pass in custom W&B arguments

In [ ]:
from ray import tune

def ray_hp_space(trial):
    return {
        "learning_rate": tune.loguniform(1e-6, 1e-4),
        "per_device_train_batch_size": tune.choice([16, 32, 64, 128]),
    }

In [ ]:
def wandb_hp_space(trial):
    return {
        "method": "random",
        "metric": {"name": "accuracy", "goal": "maximize"},
        "parameters": {
            "learning_rate": {"distribution": "uniform", "min": 1e-6, "max": 1e-4},
            "per_device_train_batch_size": {"values": [16, 32, 64, 128]},
        },
    }

In [ ]:
from transformers import AutoModelForSequenceClassification
import numpy as np
import evaluate
from transformers import EvalPrediction

# Load the accuracy metric from the evaluate library
accuracy_metric = evaluate.load("accuracy")

def compute_metrics(eval_pred: EvalPrediction) -> dict:
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    return accuracy

def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
)

train_dataset, eval_dataset = get_datasets()

trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/tmp/ipykernel_375902/4166464530.py:15: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
best_trial = trainer.hyperparameter_search(
    direction="maximize",
    backend="wandb", #"ray",  # or "optuna", "sigopt", "wandb"
    hp_space=wandb_hp_space, #ray_hp_space, 
    n_trials=20,
)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Create sweep with ID: b6xneouv
Sweep URL: https://wandb.ai/inflaton-ai/uncategorized/sweeps/b6xneouv


wandb: Agent Starting Run: dwrdunne with config:
wandb: 	learning_rate: 5.517943384545444e-06
wandb: 	per_device_train_batch_size: 16
Run dwrdunne errored:
Traceback (most recent call last):
  File "/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/wandb/agents/pyagent.py", line 306, in _run_job
    self._function()
  File "/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/transformers/integrations/integration_utils.py", line 552, in _objective
    run = wandb.run if wandb.run else wandb.init()
                                      ^^^^^^^^^^^^
  File "/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/wandb/sdk/wandb_init.py", line 1485, in init
    wandb._sentry.reraise(e)
  File "/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/wandb/analytics/sentry.py", line 156, in reraise
    raise exc.with_traceback(sys.exc_info()[2])
  File "/home/inflaton/miniconda3/envs/fake-news/lib/python3.11/site-packages/wand